### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра информационных технологий 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ №3

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Гусейнов Вахид Азерович
##### Группа:   НБИбд-01-17

## Москва 2020
***

### Вариант № 14

Алгоритм: Eclat

День недели (поле order_dow таблицы orders): “4”

Код департамента (поле department_id таблицы products): “5”

1. При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

In [1]:
import sqlite3

conn = sqlite3.connect('instacart.db')

2. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products). Определите количество записей в полученном наборе и определите количество товаров (поле order_id таблицы order_products__train) в транзакциях набора.

In [2]:
import pandas as pd

query =\
'''
SELECT DISTINCT train.*, products.product_name
FROM order_products__train as train
JOIN orders
    ON train.order_id = orders.order_id
JOIN products
    ON train.product_id = products.product_id
WHERE orders.order_dow = 4
AND   products.department_id = 5
'''
data = pd.read_sql(query, conn)

In [3]:
data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,877974,1808,1,0,Champagne
1,1859940,15511,1,1,Draft Sake
2,3409264,2120,3,1,Sauvignon Blanc
3,1881604,10607,6,1,Prosecco
4,1881604,29509,2,1,80 Vodka Holiday Edition


3. Определите количество покупок (транзакций) для пяти наиболее популярных товаров в наборе.

In [4]:
popular = data.groupby('product_name')['order_id'].count().nlargest(5)
popular

product_name
Beer                  45
Cabernet Sauvignon    43
Sauvignon Blanc       38
Chardonnay            36
India Pale Ale        31
Name: order_id, dtype: int64

4. Постройте транзакционную базу данных для поиск ассоциативных правил из полученного набора записей таблицы order_products__train, используя в качестве идентификатора транзакции поле order_id, а в качестве названий товаров - поле product_name из таблицы products, соответствующее полю product_id.

In [5]:
transactions = []
for order in data['order_id'].unique():
    products = set(data[data['order_id'] == order]['product_name'])
    pair = list((order, products))
    transactions.append(pair)

In [6]:
for i in range(3):
    print(transactions[i])

['877974', {'Champagne'}]
['1859940', {'Draft Sake'}]
['3409264', {'Sauvignon Blanc'}]


5. Реализуйте указанный в индивидуальном задании метод построения популярных наборов предметов (Apriori/Eclat/Declat) (3 балла) или используйте метод BruteForce (0 баллов). Протестируйте корректность

In [7]:
def prep(database):
    all_items = set()
    for pair in database:
        all_items = all_items.union(pair[1])
    all_items

    res = dict()
    for item in all_items:
        res[item] = set()
        for pair in database:
            if item in pair[1]:
                res[item].add(pair[0])

    res = list(res.items())
    return res

def eclat(prefix, items):
    while items:
        i, itids = items.pop()
        isupp = len(itids)
        if isupp >= minsup:
            items_freq[frozenset(prefix + [i])] = isupp
            suffix = []
            for j, ojtids in items:
                jtids = itids&ojtids
                if len(jtids)>=minsup:
                    suffix.append((j,jtids))
            eclat(prefix+[i], sorted(suffix, key=lambda item: len(item[1]), reverse=True))

Проверка

In [8]:
D_train = [
    [ 1, {"A","B","D","E"} ],
    [ 2, {"B","C","E"} ],
    [ 3, {"A","B","D","E"} ],
    [ 4, {"A","B","C","E"} ],
    [ 5, {"A","B","C","D","E"} ],
    [ 6, {"B","C","D"} ],
]

minsup = 3
items_freq = dict()
eclat([], prep(D_train))

res_lst = []
for key, value in items_freq.items():
    tmp = []
    tmp.append(tuple(key))
    tmp.append(value)
    res_lst.append(tmp)
res_lst = sorted(res_lst, key=lambda x: len(x[0]))
    
for result in res_lst:
    print(result)

[('E',), 5]
[('D',), 4]
[('B',), 6]
[('A',), 4]
[('C',), 4]
[('D', 'E'), 3]
[('C', 'E'), 3]
[('A', 'E'), 4]
[('B', 'E'), 5]
[('A', 'D'), 3]
[('D', 'B'), 4]
[('A', 'B'), 4]
[('C', 'B'), 4]
[('A', 'D', 'E'), 3]
[('B', 'D', 'E'), 3]
[('B', 'C', 'E'), 3]
[('B', 'A', 'E'), 4]
[('A', 'D', 'B'), 3]
[('B', 'A', 'D', 'E'), 3]


6. При помощи указанного в индивидуальном задании метода или метода BruteForce постройте популярные наборы товаров с минимальной поддержкой, равной половине среднего количества покупок пяти наиболее популярных товаров. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов товаров оставьте в наборе данных транзакции с 10 наиболее популярными товарами и сокращайте число записей (например, методом деления пополам).

In [9]:
minsup = round(popular.mean() / 2)
minsup

19

Возьмем minsup = 2 иначе не получим никаких ассоциативных правил

In [10]:
minsup = 2
items_freq = dict()
eclat([], prep(transactions))

res_lst = []
for key, value in items_freq.items():
    tmp = []
    tmp.append(tuple(key))
    tmp.append(value)
    res_lst.append(tmp)
res_lst = sorted(res_lst, key=lambda x: len(x[0]))
    
print('Полученные популярные наборы:\n')
for result in res_lst:
    print(result)

Полученные популярные наборы:

[('Sauvignon Blanc Wine',), 3]
[('India Pale Ale Racer 5',), 4]
[('Sonoma Pinot Noir Wine',), 2]
[('Sauvignon Blanc',), 38]
[('Old Rasputin Stout Beer',), 3]
[('12 Oz Lager',), 3]
[('Reserve Shiraz Wine',), 2]
[('Draft Sake',), 2]
[('Verry Special Cognac',), 2]
[("Vintner's Reserve Chardonnay",), 2]
[('Champagne',), 2]
[('Original Tequila Reposado',), 2]
[('Lager',), 5]
[('Bitters Liqueur',), 3]
[('Original Spiced Rum',), 2]
[('Brut Champagne',), 2]
[('Vodka',), 27]
[('Chardonnay',), 36]
[('India Pale Ale',), 31]
[('Pinot Noir, California',), 2]
[('Prosecco Sparkling Wine',), 18]
[('Sparking Brut Classic Wine',), 2]
[('Brut Rosé',), 2]
[('Diamond Collection Silver Label Pinot Noir Wine',), 2]
[('Distilled London Dry Gin',), 3]
[('Beer',), 45]
[('Potato Vodka',), 3]
[('Brut Sparkling Wine',), 6]
[('Draft Beer',), 3]
[('Frontier Straight 95% Rye Mash Whiskey',), 3]
[('Liqueur',), 2]
[('Claret, Black Label, 2006',), 2]
[('Prosecco',), 8]
[('Red Wine, Dark, C

7. Для какого-либо из полученных популярных наборов товаров постройте набор ассоциативных правил.

In [11]:
sample, _ = res_lst[-3]
print(sample, len(sample))

('India Pale Ale', 'Premium Belgian Lager', 'Variety Pack Hard Cider', 'Pinot Noir Wine', 'Cabernet Sauvignon') 5


In [12]:
from itertools import chain, combinations

def ComputeSupport(X, D):
    supX = 0
    for _, itemset in D:
        if X.issubset(itemset):
            supX += 1
    return supX

def powersetk(iterable, k):
    xs = list(iterable)
    return list(chain.from_iterable(combinations(xs, n) for n in range(k, len(xs) + 1)))[:-1]

def AssociationRules(D, Z_set, minconf):
    A_rules = []
    supZ = ComputeSupport(set(Z_set), D)
    A_set = powersetk(Z_set, 1)
    while len(A_set)>0:
        X_set = A_set[-1]
        A_set.pop()
        conf = supZ / ComputeSupport(set(X_set), D)
        if conf >= minconf:
            Y_set = sorted(list(set(Z_set) - set(X_set)))
            A_rules.append([X_set, Y_set])
        else:
            for W_set in powersetk(X_set, 1):
                if W_set in A_set:
                    A_set.remove(W_set)
    return A_rules

rules = AssociationRules(transactions, sample, 0.9)
print('Полученные ассоциативные правила:\n')
for rule in rules:
    print('rule:')
    for x in rule:
        print(x)
    print()

Полученные ассоциативные правила:

rule:
('Premium Belgian Lager', 'Variety Pack Hard Cider', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['India Pale Ale']

rule:
('India Pale Ale', 'Variety Pack Hard Cider', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['Premium Belgian Lager']

rule:
('India Pale Ale', 'Premium Belgian Lager', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['Variety Pack Hard Cider']

rule:
('India Pale Ale', 'Premium Belgian Lager', 'Variety Pack Hard Cider', 'Cabernet Sauvignon')
['Pinot Noir Wine']

rule:
('India Pale Ale', 'Premium Belgian Lager', 'Variety Pack Hard Cider', 'Pinot Noir Wine')
['Cabernet Sauvignon']

rule:
('Variety Pack Hard Cider', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['India Pale Ale', 'Premium Belgian Lager']

rule:
('Premium Belgian Lager', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['India Pale Ale', 'Variety Pack Hard Cider']

rule:
('Premium Belgian Lager', 'Variety Pack Hard Cider', 'Cabernet Sauvignon')
['India Pale Ale', 'Pinot Noir Wine']

rule:

8. Для построенного набора ассоциативных правил вычислите показатели: support, confidence, lift, leverage, conviction и выведите на экран.

In [13]:
import numpy as np

N = len(transactions)
supps = []
confs = []
lifts = []
leverages = []
convictions = []
for rule in rules:
    x1 = set(rule[0])
    x2 = set(rule[1])
    supp_x1_x2 = ComputeSupport(x1.union(x2), transactions)
    supp_x1 = ComputeSupport(x1, transactions)
    supp_x2 = ComputeSupport(x2, transactions)
    conf = supp_x1_x2 / supp_x1 # confidence
    lift = supp_x1_x2 / (supp_x1 * supp_x2) # lift
    leverage = supp_x1_x2 - supp_x1 * supp_x2 # leverage
    if conf == 1:
        conv = np.inf
    else:
        conv = (1 - supp_x2) / (1 - conf) # conviction
    supps.append(supp_x1_x2)
    confs.append(conf)
    lifts.append(lift)
    leverages.append(leverage)
    convictions.append(conv)

In [14]:
for i in range(len(rules)):
    print('rule:')
    for x in rules[i]:
        print(x)
    print('Support =', supps[i])
    print('Confidence =', confs[i])
    print('Lift =', lifts[i])
    print('Leverage =', leverages[i])
    print('Conviction =', convictions[i])
    print()

rule:
('Premium Belgian Lager', 'Variety Pack Hard Cider', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['India Pale Ale']
Support = 2
Confidence = 1.0
Lift = 0.03225806451612903
Leverage = -60
Conviction = inf

rule:
('India Pale Ale', 'Variety Pack Hard Cider', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['Premium Belgian Lager']
Support = 2
Confidence = 1.0
Lift = 0.14285714285714285
Leverage = -12
Conviction = inf

rule:
('India Pale Ale', 'Premium Belgian Lager', 'Pinot Noir Wine', 'Cabernet Sauvignon')
['Variety Pack Hard Cider']
Support = 2
Confidence = 1.0
Lift = 0.125
Leverage = -14
Conviction = inf

rule:
('India Pale Ale', 'Premium Belgian Lager', 'Variety Pack Hard Cider', 'Cabernet Sauvignon')
['Pinot Noir Wine']
Support = 2
Confidence = 1.0
Lift = 0.1
Leverage = -18
Conviction = inf

rule:
('India Pale Ale', 'Premium Belgian Lager', 'Variety Pack Hard Cider', 'Pinot Noir Wine')
['Cabernet Sauvignon']
Support = 2
Confidence = 1.0
Lift = 0.023255813953488372
Leverage = -84
Convict